#[UCI Heart Disease Data](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data)(Kaggle) - klasyfikacja choroby serca



In [2]:
import kagglehub
import os
import pandas as pd

path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")

print("Path to dataset files:", path)
print(os.listdir(path))

df = pd.read_csv(os.path.join(path, "heart_disease_uci.csv"))


Using Colab cache for faster access to the 'heart-disease-data' dataset.
Path to dataset files: /kaggle/input/heart-disease-data
['heart_disease_uci.csv']


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


#Wymiary
Rekordy: 920
Kolumny: 16 cech

#Opis cech:

###Numeryczne (int64 / float64): 8 kolumn

id – unikalny identyfikator pacjenta

age – wiek pacjenta

trestbps – ciśnienie krwi w spoczynku

chol – cholesterol w mg/dl

thalach – maksymalne tętno osiągnięte

oldpeak – ST depression wywołane wysiłkiem

ca – liczba naczyń głównych pokolorowanych fluoroskopowo

num – przewidywana etykieta choroby serca


###Kategoryczne / obiektowe (object): 8 kolumn

sex – płeć

dataset – pochodzenie badania

cp – typ bólu w klatce piersiowej

fbs – cukier na czczo > 120 mg/dl (True/False)

restecg – wynik EKG w spoczynku

exang – dławica wywołana wysiłkiem

slope – nachylenie ST segment

thal – typ defektu (normal, fixed defect, reversible defect)


###Brakujące dane

trestbps – 861/920 → ok. 6.4% braków

chol – 890/920 → ok. 3.3% braków

fbs – 830/920 → ok. 9.8% braków

slope – 611/920 → ok. 33.5% braków

ca – 309/920 → ok. 66.4% braków

thal – 434/920 → ok. 52.8% braków

In [4]:
n_rows, n_cols = df.shape
print(f"\nLiczba rekordów: {n_rows}")
print(f"Liczba cech (kolumn): {n_cols}")

print("\n--- Przykładowe wiersze ---")
print(df.head())

print("\n--- Statystyki opisowe ---")
print(df.describe())



Liczba rekordów: 920
Liczba cech (kolumn): 16

--- Przykładowe wiersze ---
   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable de

In [5]:
import numpy as np

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

target_col = 'num' if 'num' in df.columns else None
if target_col is not None:
    if target_col in num_cols:
        num_cols.remove(target_col)
    if target_col in cat_cols:
        cat_cols.remove(target_col)

print("num_cols:", num_cols)
print("cat_cols:", cat_cols)

num_cols: ['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
cat_cols: ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']


In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

num_df = df[num_cols].copy()
cat_df = df[cat_cols].copy()

half = len(num_cols) // 2
num_easy = num_cols[:half]
num_hard = num_cols[half:]

simple_imputer = SimpleImputer(strategy='mean')
num_easy_imputed = pd.DataFrame(simple_imputer.fit_transform(df[num_easy]), columns=num_easy)

knn_imputer = KNNImputer(n_neighbors=5)
num_hard_imputed = pd.DataFrame(knn_imputer.fit_transform(df[num_hard]), columns=num_hard)

num_imputed = pd.concat([num_easy_imputed, num_hard_imputed], axis=1)

cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputed = pd.DataFrame(cat_imputer.fit_transform(cat_df), columns=cat_cols)

df_imputed = pd.concat([num_imputed, cat_imputed], axis=1)

print("✅ Dane uzupełnione:")
print(df_imputed.info())

✅ Dane uzupełnione:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    float64
 1   age       920 non-null    float64
 2   trestbps  920 non-null    float64
 3   chol      920 non-null    float64
 4   thalch    920 non-null    float64
 5   oldpeak   920 non-null    float64
 6   ca        920 non-null    float64
 7   sex       920 non-null    object 
 8   dataset   920 non-null    object 
 9   cp        920 non-null    object 
 10  fbs       920 non-null    object 
 11  restecg   920 non-null    object 
 12  exang     920 non-null    object 
 13  slope     920 non-null    object 
 14  thal      920 non-null    object 
dtypes: float64(7), object(8)
memory usage: 107.9+ KB
None


In [7]:
from sklearn.preprocessing import OrdinalEncoder

df_encoded = df_imputed.copy()

ordinal_features = {
    'cp': ['asymptomatic', 'non-anginal', 'atypical angina', 'typical angina'],
    'restecg': ['normal', 'st-t abnormality', 'lv hypertrophy'],
    'slope': ['upsloping', 'flat', 'downsloping'],
    'thal': ['normal', 'reversable defect', 'fixed defect']
}

encoder = OrdinalEncoder(categories=[ordinal_features[col] for col in ordinal_features])
df_encoded[list(ordinal_features.keys())] = encoder.fit_transform(df_encoded[list(ordinal_features.keys())])

nominal_features = ['sex', 'dataset', 'fbs', 'exang']
df_encoded = pd.get_dummies(df_encoded, columns=nominal_features, drop_first=True)

print("✅ Wszystkie kolumny zakodowane numerycznie:")
print(df_encoded.info())
df_encoded.head()


✅ Wszystkie kolumny zakodowane numerycznie:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     920 non-null    float64
 1   age                    920 non-null    float64
 2   trestbps               920 non-null    float64
 3   chol                   920 non-null    float64
 4   thalch                 920 non-null    float64
 5   oldpeak                920 non-null    float64
 6   ca                     920 non-null    float64
 7   cp                     920 non-null    float64
 8   restecg                920 non-null    float64
 9   slope                  920 non-null    float64
 10  thal                   920 non-null    float64
 11  sex_Male               920 non-null    bool   
 12  dataset_Hungary        920 non-null    bool   
 13  dataset_Switzerland    920 non-null    bool   
 14  dataset_VA Lon

,id,age,trestbps,chol,thalch,oldpeak,ca,cp,restecg,slope,thal,sex_Male,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach,fbs_True,exang_True
0,1.0,63.0,145.0,233.0,150.0,2.3,0.0,3.0,2.0,2.0,2.0,True,False,False,False,True,False
1,2.0,67.0,160.0,286.0,108.0,1.5,3.0,0.0,2.0,1.0,0.0,True,False,False,False,False,True
2,3.0,67.0,120.0,229.0,129.0,2.6,2.0,0.0,2.0,1.0,1.0,True,False,False,False,False,True
3,4.0,37.0,130.0,250.0,187.0,3.5,0.0,1.0,0.0,2.0,0.0,True,False,False,False,False,False
4,5.0,41.0,130.0,204.0,172.0,1.4,0.0,2.0,2.0,0.0,0.0,False,False,False,False,False,False


## 🛠️ Przetwarzanie danych — imputacja i kodowanie

1. **Imputacja braków**  
   - Dane numeryczne podzielono na dwa podzbiory:
     - Podzbiór A: imputacja prostą metodą (średnia)  
     - Podzbiór B: imputacja bardziej zaawansowaną metodą (KNNImputer)  
   - Dla danych kategorycznych zastosowano imputację wartościami najczęstszymi (`most_frequent`).

2. **Kodowanie zmiennych kategorycznych**  
   - Zmienne z naturalnym porządkiem (np. `cp`, `restecg`, `slope`, `thal`) zakodowano za pomocą **OrdinalEncoder**, by zachować sens kolejności.  
   - Pozostałe zmienne nominalne (np. `sex`, `dataset`, `fbs`, `exang`) zakodowano metodą **One-Hot Encoding**, usuwając jedną referencyjną kategorię (drop_first=True), aby uniknąć współliniowości.

Po tych operacjach wszystkie cechy mają typ numeryczny (float lub bool), bez brakujących wartości — dane są gotowe do analizy statystycznej i modelowania.  


### 🔄 Zmodyfikowane i zakodowane cechy

Podczas przygotowania danych kilka kolumn kategorycznych zostało przekształconych na formę numeryczną:

1. **`sex`** → zastąpiono przez kolumnę binarną `sex_Male`  
   - Wartość `True` oznacza mężczyznę, `False` – kobietę.  

2. **`dataset`** → zastąpiono trzema kolumnami logicznymi:  
   - `dataset_Hungary`, `dataset_Switzerland`, `dataset_VA Long Beach`  
   - Brak wartości w tych kolumnach (wszystkie `False`) oznacza dane z **Cleveland** (usunięta referencja po One-Hot Encoding).

3. **`cp` (chest pain type)** → zakodowano liczbowo metodą **OrdinalEncoder**:  
   - 0 – typical angina  
   - 1 – atypical angina  
   - 2 – non-anginal  
   - 3 – asymptomatic  

4. **`restecg`** → zakodowano liczbowo metodą **OrdinalEncoder**:  
   - 0 – normal  
   - 1 – stt abnormality  
   - 2 – lv hypertrophy  

5. **`slope`** → zakodowano liczbowo metodą **OrdinalEncoder**:  
   - 0 – upsloping  
   - 1 – flat  
   - 2 – downsloping  

6. **`thal`** → zakodowano liczbowo metodą **OrdinalEncoder**:  
   - 0 – normal  
   - 1 – fixed defect  
   - 2 – reversible defect  

7. **`fbs` i `exang`** → zastąpione przez kolumny binarne (`fbs_True`, `exang_True`)  
   - Wartość `True` oznacza odpowiednio: cukier na czczo >120 mg/dl / wystąpienie dławicy przy wysiłku.

---

Wszystkie pozostałe kolumny (`age`, `trestbps`, `chol`, `thalch`, `oldpeak`, `ca`) zachowały oryginalną postać liczbową (float), jedynie uzupełniono w nich brakujące wartości.


In [17]:
display(df_encoded.describe().T)

,count,mean,std,min,25%,50%,75%,max
id,920.0,460.500000,265.725422,1.0,230.75,460.5,690.25,920.0
age,920.0,53.510870,9.424685,28.0,47.00,54.0,60.00,77.0
trestbps,920.0,132.132404,18.443895,0.0,120.00,130.0,140.00,200.0
chol,920.0,198.970642,109.467013,0.0,175.00,223.0,267.00,603.0
thalch,920.0,137.741997,25.359888,60.0,120.00,140.0,157.00,202.0
oldpeak,920.0,0.877083,1.062098,-2.6,0.00,0.6,1.50,6.2
ca,920.0,0.766553,0.674078,0.0,0.20,0.8,1.20,3.0
cp,920.0,0.750000,0.930969,0.0,0.00,0.0,1.00,3.0
restecg,920.0,0.603261,0.805443,0.0,0.00,0.0,1.00,2.0
slope,920.0,0.847826,0.516007,0.0,1.00,1.0,1.00,2.0


In [24]:
# 2️⃣ Rozkład wieku pacjentów + średni cholesterol w tych przedziałach
print("\n=== Rozkład wieku pacjentów ===")
age_bins = pd.cut(df_encoded['age'], bins=5)
age_distribution = df_encoded['age'].value_counts(bins=5).sort_index()

print("Liczba pacjentów w przedziałach wiekowych:")
print(age_distribution)

print("\nŚredni cholesterol (chol) w tych samych przedziałach wiekowych:")
print(df_encoded.groupby(age_bins)['chol'].mean().round(2))



=== Rozkład wieku pacjentów ===
Liczba pacjentów w przedziałach wiekowych:
(27.95, 37.8]     49
(37.8, 47.6]     190
(47.6, 57.4]     350
(57.4, 67.2]     277
(67.2, 77.0]      54
Name: count, dtype: int64

Średni cholesterol (chol) w tych samych przedziałach wiekowych:
age
(27.951, 37.8]    211.62
(37.8, 47.6]      211.81
(47.6, 57.4]      203.94
(57.4, 67.2]      186.34
(67.2, 77.0]      174.87
Name: chol, dtype: float64


/tmp/ipython-input-1425173653.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_encoded.groupby(age_bins)['chol'].mean().round(2))


In [19]:
# 3️⃣ Średnie wartości ciśnienia i cholesterolu w zależności od kraju pochodzenia
print("\n=== Średnie ciśnienie i cholesterol w zależności od kraju ===")
country_cols = ['dataset_Hungary', 'dataset_Switzerland', 'dataset_VA Long Beach']
df_encoded['dataset_origin'] = df_encoded[country_cols].idxmax(axis=1).replace({
    'dataset_Hungary': 'Hungary',
    'dataset_Switzerland': 'Switzerland',
    'dataset_VA Long Beach': 'VA Long Beach'
})
df_encoded.loc[(df_encoded[country_cols] == False).all(axis=1), 'dataset_origin'] = 'Cleveland'
print(df_encoded.groupby('dataset_origin')[['trestbps', 'chol']].mean().round(2))


=== Średnie ciśnienie i cholesterol w zależności od kraju ===
                trestbps    chol
dataset_origin                  
Cleveland         131.68  246.32
Hungary           132.59  247.19
Switzerland       130.24    0.00
VA Long Beach     133.31  178.73


In [21]:
print("\n=== Średnie tętno maksymalne (thalch) w zależności od płci ===")
print(df_encoded.groupby('sex_Male')['thalch'].mean().rename({True: "Mężczyźni", False: "Kobiety"}))


=== Średnie tętno maksymalne (thalch) w zależności od płci ===
sex_Male
Kobiety      146.135052
Mężczyźni    135.499225
Name: thalch, dtype: float64


In [22]:
# 5️⃣ Częstość występowania dławicy wysiłkowej (exang) w zależności od płci
print("\n=== Dławica wysiłkowa (exang_True) w zależności od płci ===")
print(df_encoded.groupby('sex_Male')['exang_True'].mean().rename({True: "Mężczyźni", False: "Kobiety"}))


=== Dławica wysiłkowa (exang_True) w zależności od płci ===
sex_Male
Kobiety      0.221649
Mężczyźni    0.404959
Name: exang_True, dtype: float64


In [28]:
print("\n=== Korelacje między wybranymi zmiennymi ===")
corr_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
print(df_encoded[corr_cols].corr().round(2))


=== Korelacje między wybranymi zmiennymi ===
           age  trestbps  chol  thalch  oldpeak
age       1.00      0.24 -0.09   -0.35     0.25
trestbps  0.24      1.00  0.09   -0.10     0.16
chol     -0.09      0.09  1.00    0.24     0.05
thalch   -0.35     -0.10  0.24    1.00    -0.15
oldpeak   0.25      0.16  0.05   -0.15     1.00


In [26]:
if 'num' not in df_encoded.columns and 'num' in df.columns:
    df_encoded['num'] = df['num']


In [27]:
print("=== Średni wiek pacjentów z chorobą i bez choroby, z podziałem na płeć ===")

# Obliczamy średni wiek pacjentów z chorobą i bez choroby (num > 0 oznacza obecność choroby)
age_by_sex_disease = df_encoded.groupby(['sex_Male', df_encoded['num'] > 0])['age'].mean().unstack()
age_by_sex_disease.columns = ['Brak choroby', 'Choroba serca']
age_by_sex_disease.index = ['Kobiety', 'Mężczyźni']
print(age_by_sex_disease.round(1))

# Udział przypadków choroby w każdej grupie płci
print("\n=== Odsetek osób z chorobą serca w zależności od płci ===")
disease_rate_by_sex = df_encoded.groupby('sex_Male')['num'].apply(lambda x: (x > 0).mean())
disease_rate_by_sex.index = ['Kobiety', 'Mężczyźni']
print((disease_rate_by_sex * 100).round(1).astype(str) + '%')


=== Średni wiek pacjentów z chorobą i bez choroby, z podziałem na płeć ===
           Brak choroby  Choroba serca
Kobiety            51.2           56.2
Mężczyźni          50.2           55.9

=== Odsetek osób z chorobą serca w zależności od płci ===
Kobiety      25.8%
Mężczyźni    63.2%
Name: num, dtype: object
